In [67]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import keras
datas1=pd.read_csv('C:/Users/USER/Documents/python projects/sentiment/amazon_cells_labelled.txt',delimiter='\t',header=None)
datas2=pd.read_csv('C:/Users/USER/Documents/python projects/sentiment/yelp_labelled.txt',delimiter='\t',header=None)
datas3=pd.read_csv('C:/Users/USER/Documents/python projects/sentiment/imdb_labelled.txt',delimiter='\t',header=None)

data=pd.concat([datas1,datas2,datas3])

In [68]:
data

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0


In [69]:
data[0]=data[0].str.lower()


In [70]:
data.loc[(data[1] ==0),1] = 'negative'

In [71]:
data.loc[(data[1]==1),1]='positive'

In [72]:
data

,0,1
0,so there is no way for me to plug it in here i...,negative
1,"good case, excellent value.",positive
2,great for the jawbone.,positive
3,tied to charger for conversations lasting more...,negative
4,the mic is great.,positive
...,...,...
743,i just got bored watching jessice lange take h...,negative
744,"unfortunately, any virtue in this film's produ...",negative
745,"in a word, it is embarrassing.",negative
746,exceptionally bad!,negative


In [73]:
x=data[0]
y=data[1]

In [74]:
def cleanLower(texts):
    
    # This regular expression pattern means everything except alphabetical characters
    pattern = "^[a-zA-Z]"
    cleanText = []
    for text in texts:
        # re.sub(pattern) means replace everything with a space except alphabetical characters
        cleanText.append(re.sub(pattern," ",text).lower())
        
    return cleanText

x = cleanLower(x)

In [75]:
x[1]

' ood case, excellent value.'

In [76]:

max_words = 50000
max_len = 100

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(list(data[0]))
train_data = tokenizer.texts_to_sequences(list(data[0]))

In [77]:
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data,maxlen = max_len)

In [78]:
train_data

array([[   0,    0,    0, ...,   62,    4, 2267],
       [   0,    0,    0, ...,  159,   93,  527],
       [   0,    0,    0, ...,   12,    1, 1094],
       ...,
       [   0,    0,    0, ...,    6,    5,  779],
       [   0,    0,    0, ...,    0, 5270,   48],
       [   0,    0,    0, ...,  154,    9,  161]])

In [79]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
le = preprocessing.LabelEncoder()

In [80]:
le.fit(y)

LabelEncoder()

In [81]:
list(le.classes_)

['negative', 'positive']

In [82]:
y2=le.fit_transform(y)
total_languages = data[1].nunique()
y2 = keras.utils.to_categorical(y2,num_classes=total_languages)

In [83]:
X_train,X_test,Y_train,Y_test = train_test_split(train_data,y2)

embedding_dims = 500
vocab_size = len(tokenizer.word_index)+1

In [84]:
y2

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [85]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim = vocab_size,output_dim = embedding_dims,input_length = max_len),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(total_languages,activation=tf.nn.softmax)
                            ])


model.compile(optimizer ='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [86]:
model.fit(X_train,Y_train)

65/65 [==============================] - 2s 37ms/step - loss: 0.7456 - accuracy: 0.5425


In [87]:
model_predict=model.predict(X_test)

In [91]:
text = ["very bad movie"]
test_text = tokenizer.texts_to_sequences(text)
test_text = tf.keras.preprocessing.sequence.pad_sequences(test_text, maxlen=max_len)

In [92]:

predictions = model.predict(test_text)

In [93]:
out = predictions.argmax()
print(le.inverse_transform([out]))


['negative']
